In [9]:
# ============================================================
# Semantic vs Brain Similarity Analysis (Weighted RSA version)
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# --- Prepare output folders ---
os.makedirs("results", exist_ok=True)
os.makedirs("figures", exist_ok=True)

# --- Load label data ---
labels = np.load("../data/kay_labels.npy", allow_pickle=True)
if labels.shape[0] == 4:
    labels = labels.T
labels = labels.astype(str)
label_data = labels[:, :4]  # 4 semantic hierarchy levels

# --- Compute weighted label similarity matrix ---
n_images = label_data.shape[0]
weights = [4, 3, 2, 1]  # total match = 10
label_similarity = np.zeros((n_images, n_images))

for i in range(n_images):
    for j in range(n_images):
        score = sum(weights[k] for k in range(4) if label_data[i, k] == label_data[j, k])
        label_similarity[i, j] = score

# --- Load fMRI data ---
data = np.load("../data/kay_images.npz")
responses = data["responses"]
roi = data["roi"]
roi_names = data["roi_names"]

# --- Run RSA across ROIs ---
results = {}
iu = np.triu_indices(n_images, k=1)  # upper triangle indices

for roi_idx, roi_name in enumerate(roi_names):
    # Mask voxels for this ROI
    if roi.ndim == 1:
        voxel_mask = (roi == roi_idx)
    else:
        voxel_mask = (roi[0] == roi_idx)

    if voxel_mask.sum() == 0:
        print(f"Skipping {roi_name} (no voxels)")
        continue

    responses_roi = responses[:, voxel_mask]
    brain_corr = np.corrcoef(responses_roi)

    flat_label_sim = label_similarity[iu]
    flat_brain_corr = brain_corr[iu]

    # Compute Pearson correlation (semantic vs neural)
    rho = np.corrcoef(flat_label_sim, flat_brain_corr)[0, 1]
    results[roi_name] = rho

    # --- Plot scatter and save ---
    plt.figure(figsize=(6, 4))
    plt.scatter(flat_label_sim, flat_brain_corr, alpha=0.25, color='steelblue', edgecolors='none')
    plt.xlabel("Semantic Similarity (Weighted)")
    plt.ylabel("Brain Similarity (Pearson)")
    plt.title(f"{roi_name} — Semantic vs Brain Similarity")
    plt.grid(True, linestyle=':')
    plt.tight_layout()
    plt.savefig(f"figures/{roi_name}_scatter.png", dpi=300)
    plt.close()

# --- Save all results ---
df = pd.DataFrame(list(results.items()), columns=["ROI", "Pearson_r"])
df.to_csv("results/rsa_weighted.csv", index=False)

print("✅ RSA analysis completed.")
print(df)
print("\nFigures saved in /figures and results in /results ✅")

Skipping Other (no voxels)
✅ RSA analysis completed.
      ROI  Pearson_r
0      V1   0.002390
1      V2   0.004369
2      V3   0.005982
3     V3A   0.012207
4     V3B   0.023338
5      V4   0.016807
6  LatOcc   0.038627

Figures saved in /figures and results in /results ✅
